In [1]:
!pip install qwen_vl_utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.7/38.7 MB 21.1 MB/s eta 0:00:00


In [2]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info


model = Qwen2VLForConditionalGeneration.from_pretrained(
    "prithivMLmods/Qwen2-VL-OCR-2B-Instruct", torch_dtype="auto", device_map="auto"
)

processor = AutoProcessor.from_pretrained("prithivMLmods/Qwen2-VL-OCR-2B-Instruct")


messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://ai-bot-builder.heltar.com/api/typebots/cm1hq6zrr0034zr0c4v2twxxr/whatsapp/media/2323296558046107",
            },
            {"type": "text", "text": "Describe this image."},
        ],
    }
]


text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.42G [00:00<?, ?B/s]

`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


generation_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/408 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

['The image is a professional invoice template. The invoice is for East Repair Inc., located at 1912 Harvest Lane, New York, NY 12210. The invoice is dated 11/02/2019 and includes the following details:\n\n- **Bill to**: John Smith\n- **Ship to**: John Smith\n- **Invoice #**: US-001\n- **Invoice Date**: 11/02/2019\n- **P.O. #**: 2312/2019\n- **Due Date**: 26']


In [3]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "https://ai-bot-builder.heltar.com/api/typebots/cm1hq6zrr0034zr0c4v2twxxr/whatsapp/media/2323296558046107",
            },
            {"type": "text", "text": "Extract all the details in JSON format."},
        ],
    }
]


text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")


generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)

['```json\n{\n  "invoice": {\n    "company": "East Repair Inc.",\n    "address": {\n      "street": "1912 Harvest Lane",\n      "city": "New York",\n      "state": "NY",\n      "zip": "12210"\n    },\n    "to": {\n      "name": "John Smith",\n      "address": {\n        "street": "2 Court Square",\n        "city": "New York",\n        "state": "NY",\n        "zip": "12210"\n      }\n    },\n    "from": {\n      "name']
